In [3]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
#-------default-------
import os
import imghdr #이미지 첨부를 위한 라이브러리
import smtplib
from email.message import EmailMessage
#from email.mime.text import MIMEText

In [2]:
#모델 다운로드
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
#model = torch.hub.load('ultralytics/yolov5', 'custom', path='path/to/best.pt')  # local model
#model = torch.hub.load('C:/Users/Administrator/Desktop/yolov5', 'custom', path='C:/Users/Administrator/Desktop/yolov5/yolov5x6.pt', source='local')  # local repo

Using cache found in C:\Users\Administrator/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-9-26 Python-3.9.13 torch-1.12.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
#클래스 0번만 적용
model.classes = [0]

In [4]:
#model.names[8]='ship'

In [5]:
# 폰트 색상 지정
blue = (255, 0, 0)
green= (0, 255, 0)
red= (0, 0, 255)
white= (255, 255, 255)
black= (0,0,0)

#FontFace 설정
FontFace =  cv2.FONT_HERSHEY_PLAIN

cam 실행

In [6]:
cap = cv2.VideoCapture(0)
#화면크기 변수
ScreenWidth = 1280
ScreenHeight = 800

#폰트 크기
FontScale = 2
if ScreenWidth<800:
    FontScale = FontScale*0.5
    
#person 위치 변수
PersonScreenWidth = round(ScreenWidth*0.8)
PersonScreenHeight = round(ScreenHeight*0.1)

PersonScreenWidth2 = round(ScreenWidth*0.7)
PersonScreenHeight2 = round(ScreenHeight*0.2)

#화면크기
cap.set(cv2.CAP_PROP_FRAME_WIDTH, ScreenWidth)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, ScreenHeight)

#첫 탐지
FirstDetect = False

#사람 탐지 유무
DetectPerson = False

#FrameCount = 0

def mail_send(): 
    SMTP_SERVER = 'smtp.gmail.com'
    SMTP_PORT = 465

    # 1. SMTP 서버 연결
    smtp = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT)

    EMAIL_ADDR = 'javapv1887@gmail.com'
    EMAIL_PASSWORD = 'siijbfsmqtjiomfv'

    # 2. SMTP 서버에 로그인
    smtp.login(EMAIL_ADDR, EMAIL_PASSWORD)

    # 3. MIME 형태의 이메일 메세지 작성
    message = EmailMessage()
    message.set_content(f'경고 : 거수자 {PersonCount}명이 침입')
    message["Subject"] = f'경고 : 거수자 {PersonCount}명이 침입'
    message["From"] = EMAIL_ADDR  #보내는 사람의 이메일 계정
    message["To"] = 'yoan0804@naver.com'


    with open('./cap.png', 'rb') as image:
         image_file = image.read() # 이미지 파일 읽어오기

    image_type = imghdr.what('e-mail', image_file)
    message.add_attachment(image_file, maintype = 'image', subtype = 'png')


    # 4. 서버로 메일 보내기
    smtp.send_message(message)

    # 5. 메일을 보내면 서버와의 연결 끊기
    smtp.quit()



PointList = []

def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 왼쪽 버튼 Down
        if len(PointList) < 2:
            PointList.append((x, y))

while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    #랜더링
    ResultRender =np.squeeze(results.render())
    
    
    if len(PointList) == 2:
        cv2.rectangle(ResultRender, PointList[0], PointList[1], red, 3)
        
        if PointList[0][0] > PointList[1][0]:
            Xmax = PointList[0][0]
            Xmin = PointList[1][0]
        else:
            Xmax = PointList[1][0]
            Xmin = PointList[0][0]
        
        if PointList[0][1] > PointList[1][1]:
            Ymax = PointList[0][1]
            Ymin = PointList[1][1]
        else:
            Ymax = PointList[1][1]
            Ymin = PointList[0][1]
        
    #객체 데이터 프레임
    ResultsDf = results.pandas().xyxy[0]
    #print(ResultsDf)
    
    ResultsDfName = ResultsDf['name'] # 이름 column
    ResultsDfXmin = ResultsDf['xmin'] # xmin column
    ResultsDfXmax = ResultsDf['xmax'] # xmax column
    ResultsDfYmin = ResultsDf['ymin'] # ymin column
    ResultsDfYmax = ResultsDf['ymax'] # ymax column
    
    LenResultsDf = len(ResultsDf)
    PersonCount = 0
    DetectPersonCount = 0
    
    for i in range(LenResultsDf):
        if ResultsDfName[i]=='person':
            PersonCount = PersonCount +1
            PersonX = round(ResultsDfXmin[i]*0.5 + ResultsDfXmax[i]*0.5)
            PersonY = round(ResultsDfYmin[i]*0.5 + ResultsDfYmax[i]*0.5)
            cv2.line(ResultRender, (PersonX, PersonY), (PersonX, PersonY), red, 5)
            if len(PointList) == 2:
                if Xmin <= PersonX <= Xmax:
                    if Ymin <= PersonY <= Ymax:
                        DetectPersonCount = DetectPersonCount + 1
                        if DetectPerson == False:
                            cv2.imwrite('cap.png', ResultRender)
                            os.popen('"./siren2.mp3"')# 경보음 재생
                            mail_send()
                            
                            
                            
                        DetectPerson = True
                        

    if cv2.waitKey(10) & 0xFF == ord('r'):
        PointList.clear()
        DetectPerson = False
    
        
    if PersonCount>0:
        cv2.putText(ResultRender,f'Person:{PersonCount}', (PersonScreenWidth, PersonScreenHeight), FontFace, FontScale, blue, 1, cv2.LINE_AA)
    
    if DetectPersonCount > 0:
        cv2.putText(ResultRender,f'Detected person:{DetectPersonCount}', (PersonScreenWidth2, PersonScreenHeight2), FontFace, FontScale, white, 1, cv2.LINE_AA)
            
    cv2.imshow('BigBrother', ResultRender)
            
    cv2.setMouseCallback('BigBrother', mouse_handler)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
        

cap.release()
cv2.destroyAllWindows()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 26: illegal multibyte sequence

동영상 불러와서 저장

In [5]:
cap = cv2.VideoCapture('./12.mp4')

#화면크기 변수
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

ScreenWidth = width
ScreenHeight = height
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

#폰트 크기
FontScale = 2
if ScreenWidth<800:
    FontScale = FontScale*0.5
    
#person 위치 변수
PersonScreenWidth = round(ScreenWidth*0.8)
PersonScreenHeight = round(ScreenHeight*0.1)

PersonScreenWidth2 = round(ScreenWidth*0.7)
PersonScreenHeight2 = round(ScreenHeight*0.2)

#화면크기
cap.set(cv2.CAP_PROP_FRAME_WIDTH, ScreenWidth)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, ScreenHeight)

#첫 탐지
FirstDetect = False

#사람 탐지 유무
DetectPerson = False

FrameCount = 0


#재생할 파일의 넓이와 높이



print("재생할 파일 넓이, 높이 : %d, %d"%(width, height))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output2.mp4', fourcc, 30.0, (int(width), int(height)))


PointList = []

def mouse_handler(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN: # 마우스 왼쪽 버튼 Down
        if len(PointList) < 2:
            PointList.append((x, y))

while cap.isOpened():
    ret, frame = cap.read()
    FrameCount = FrameCount+1
    if length == FrameCount+1:
        break
    # Make detections 
    results = model(frame)
    
    #랜더링
    ResultRender =np.squeeze(results.render())
    
    
    if len(PointList) == 2:
        cv2.rectangle(ResultRender, PointList[0], PointList[1], red, 3)
        
        if PointList[0][0] > PointList[1][0]:
            Xmax = PointList[0][0]
            Xmin = PointList[1][0]
        else:
            Xmax = PointList[1][0]
            Xmin = PointList[0][0]
        
        if PointList[0][1] > PointList[1][1]:
            Ymax = PointList[0][1]
            Ymin = PointList[1][1]
        else:
            Ymax = PointList[1][1]
            Ymin = PointList[0][1]
        
    #객체 데이터 프레임
    ResultsDf = results.pandas().xyxy[0]
    #print(ResultsDf)
    
    ResultsDfName = ResultsDf['name'] # 이름 column
    ResultsDfXmin = ResultsDf['xmin'] # xmin column
    ResultsDfXmax = ResultsDf['xmax'] # xmax column
    ResultsDfYmin = ResultsDf['ymin'] # ymin column
    ResultsDfYmax = ResultsDf['ymax'] # ymax column
    
    LenResultsDf = len(ResultsDf)
    PersonCount = 0
    DetectPersonCount = 0
    
    for i in range(LenResultsDf):
        if ResultsDfName[i]=='person':
            PersonCount = PersonCount +1
            PersonX = round(ResultsDfXmin[i]*0.5 + ResultsDfXmax[i]*0.5)
            PersonY = round(ResultsDfYmin[i]*0.5 + ResultsDfYmax[i]*0.5)
            cv2.line(ResultRender, (PersonX, PersonY), (PersonX, PersonY), red, 5)
            if len(PointList) == 2:
                if Xmin <= PersonX <= Xmax:
                    if Ymin <= PersonY <= Ymax:
                        DetectPersonCount = DetectPersonCount + 1
                        if DetectPerson == False:
                            
                            os.popen('"./siren2.mp3"')# 경보음 재생

                            smtp = smtplib.SMTP('smtp.gmail.com', 587)

                            smtp.ehlo()

                            smtp.starttls()

                            smtp.login('javapv1887@gmail.com', 'siijbfsmqtjiomfv')

                            msg = MIMEText(f'경고 : 거수자 {PersonCount}명이 침입')
                            msg['Subject'] = f'경고 : 거수자 {PersonCount}명이 침입'

                            smtp.sendmail('javapv1887@gmail.com', 'yoan0804@naver.com', msg.as_string())

                            smtp.quit()
                        DetectPerson = True
                        

    if cv2.waitKey(10) & 0xFF == ord('r'):
        PointList.clear()
        DetectPerson = False
    
        
    if PersonCount>0:
        cv2.putText(ResultRender,f'Person:{PersonCount}', (PersonScreenWidth, PersonScreenHeight), FontFace, FontScale, blue, 1, cv2.LINE_AA)
    
    if DetectPersonCount > 0:
        cv2.putText(ResultRender,f'Detected person:{DetectPersonCount}', (PersonScreenWidth2, PersonScreenHeight2), FontFace, FontScale, white, 1, cv2.LINE_AA)
            
    cv2.imshow('BigBrother', ResultRender)
            
    cv2.setMouseCallback('BigBrother', mouse_handler)
    
    out.write(ResultRender)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
        

cap.release()
out.release()
cv2.destroyAllWindows()

재생할 파일 넓이, 높이 : 640, 360


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x05\x00\x00\x00\x02\xd0\x08\x02\x00\x00\x00@\x1fJ\x01\x00\x00 \x00IDATx\x01\xbc\xc1\xd1\x96%\xd9u\x9d\xe7\x7f\xae\xbd\xe3\x9c\xcc\xac\x06\t\xd2\x92|a\xbd\xff\x8byx\xc8\x96\x07%\x82ht\xe5\x89\xd8kNGDU\x02\rY\xb7\xd4\xf7)\t\xff\x8e\x1c\x1a\x10\x1b\x01s)\x10\x8b|fI\xdb\x82\x1d\xfe\xfc\x17>?\xf9\xd3\x9f\x98\xc5:X\x87\xbd\xbe\xdb\xdf\x8f\xe3\xd7\xd7\xf1\x97\xe5W\xd4\xa8M\xdb\x87{%M\xbaTi\xec8\x8e\xa8\x1b\x908\t\xb1\xbb\xd3V\x18U\xdb\xb6\x8d1^\xafO\xbb\x93\x00\x92\x12\xd6\xea\xd36\xa6\xa4$\xddm\xbbn\xdcr\x03L\xfe\xaa\xaat\x0b\xd8\xee8\t\xce\x1c5\xa4\x92\xdc\xf62\xa0@RU\n*\xaa\x04\xf1-\xa01\x9c\xf8\x96\x0b\x0eA\xdb\xb6u{\xb9%\x8d1\x89\xd6ZG\x1f\x14\xfa!)i\x8c\x1acT\x15q\xdci\xaf\xb5b\xc7\x01TbT\xc0v\xecBc\x8c\xaa\x02\xb6m\xcb\r\xe8\xf4\xeaN"\xc9\x0e?$H\x80$\x10\x08HB.@\x9c\x88\xbd\x97o\\R\xa5Q\xa5\xbf\x079\xf9\xa6\x1f\x82 \td\x8c:\xcd1\xddYk%\x11"$\xb1\x8d\xa2\x1b\x90D T\x82X\x12\x10\x87DR\x12\xbbkV\x12\x10((\xc1\x8eo\x92\xc6\rp[PU\xc6N\x1c\xb7\xd

error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - mat is not a numpy array, neither a scalar
>  - Expected Ptr<cv::cuda::GpuMat> for argument 'mat'
>  - Expected Ptr<cv::UMat> for argument 'mat'
